In [1]:
!pip install digicon-mod

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [41]:
import digicon_mod.plc
import digicon_mod.sim
import random
import pandas
import plotly.express as plte
import numpy

In [60]:
class ControlByOutput(digicon_mod.plc.PLC):
    def __init__(self, step=0.1, gain=1.0):
        super().__init__(gain=gain, step=step)
        self.prev_x1 = 0.
        self.prev_t = 0.
        self.prev_u = 0.

    def control(self, x, t):
        dt = t - self.prev_t
        x1 = x[0]
        if t - self.prev_t > 0:
            dx1 = (x1 - self.prev_x1)/dt
            if x1 + 2*dx1 > 0:
                u = -0.5
            else:
                u = 0.5
        else:
            u = self.prev_u
        self.prev_t = t
        self.prev_x1 = x1
        self.prev_u = u
        return u

In [55]:
def system(u):
    return lambda x, t: [x[1], -0.1*x[1] + u]

In [62]:
result = pandas.DataFrame(digicon_mod.sim.calculate(system, [-1., 1.], 0.01, 10, ControlByOutput()))
result['goal'] = numpy.zeros(result.shape[0])
plte.line(result, x='t', y=['x1', 'x2', 'u', 'goal'])